In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/PyAI/16 Seq2Seq

/content/drive/MyDrive/PyAI/16 Seq2Seq


In [3]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))

import custom
import numpy
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [4]:

# [" ",1,2,3,4,5,6,7,8,9,0,+]

embedding_tensor = torch.cat([torch.zeros(1,11),torch.eye(11)])
print(embedding_tensor)


dic = {}

dic[" "] = 0
for i in range(1,10) :
    dic[str(i)] = i
dic["0"] = 10
dic["+"] = 11

print(dic)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])
{' ': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '0': 10, '+': 11}


In [5]:
with open("addition.txt", mode = "r") as f:
    ori_data = f.read()

In [6]:
data = ori_data.split("\n")

x = []
t = []
for nums in data :
    if nums.find("_") < 0 :
        continue
    nums = nums.split("_")
    x.append(nums[0])
    t.append(nums[1])

print(len(x))
print(len(x[0]))
print(list(x[0]))
print()
print(len(t))
print(len(t[0]))
print(list(t[0]))

50000
7
['1', '6', '+', '7', '5', ' ', ' ']

50000
4
['9', '1', ' ', ' ']


In [7]:
vector_x = []
for i in range(len(x)) :
    temp = list(x[i])
    vector = custom.word_vectorize(temp, dic, 7, pad_word = " ", unk_word = " ")
    vector_x.append(vector)

vector_x = numpy.array(vector_x)
print(vector_x.shape)

(50000, 7)


In [8]:
vector_t = []
for i in range(len(t)) :
    temp = list(t[i])
    vector = custom.word_vectorize(temp, dic, 4)
    vector_t.append(vector)

vector_t = numpy.array(vector_t)
print(vector_t.shape)

(50000, 4)


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(vector_x, dtype = torch.long, device = device)
tensor_t = torch.tensor(vector_t, dtype = torch.long, device = device)

train_zip_list = list(zip(tensor_x[:40000], tensor_t[:40000]))
test_zip_list = list(zip(tensor_x[40000:], tensor_t[40000:]))

train_dataloader = DataLoader(train_zip_list, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_zip_list, batch_size=1000, shuffle=False)

In [10]:
class Encoder(nn.Module):
    def __init__(self, embedding_tensor):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_tensor, freeze=True, padding_idx=0)
        self.rnn = nn.LSTM(embedding_tensor.shape[1], embedding_tensor.shape[1], batch_first=True, bidirectional=True)

    def forward(self, x):
        # x = torch.flip(x, [-1])
        x = self.embedding(x)
        output, hc = self.rnn(x)
        return output, hc

class Decoder(nn.Module) :
    def __init__(self, embedding_tensor):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_tensor, freeze=True, padding_idx=0)
        self.rnn = nn.LSTM(embedding_tensor.shape[1] * 3, embedding_tensor.shape[1], batch_first=True, bidirectional=True)
        self.f = nn.Linear(embedding_tensor.shape[1] * 4, embedding_tensor.shape[0])
        self.encoder_h_context = None

    def forward(self, encoder_output, encoder_hc, t = None) :
        encoder_h_forward = encoder_hc[0][0:1,:,:]
        encoder_h_backward = encoder_hc[0][1:2,:,:]
        self.encoder_h_context = torch.concat([encoder_h_forward,encoder_h_backward], dim = -1).transpose(0,1)
        batch_size = encoder_output.shape[0]
        decoder_input = torch.zeros(batch_size, 1).type(torch.long).to(encoder_output.device)
        decoder_hc = encoder_hc
        decoder_output_list = []

        for i in range(4) :
            decoder_output, decoder_hc = self.forward_sub(decoder_input, decoder_hc)
            decoder_output_list.append(decoder_output)

            if t is None :
                decoder_input = decoder_output.argmax(dim = -1).detach()
            else :
                decoder_input = t[:, i].unsqueeze(-1)

        decoder_output_list = torch.cat(decoder_output_list, dim=1)
        return decoder_output_list, decoder_hc, None

    def forward_sub(self, x, h) :
        x = self.embedding(x)
        x = torch.concat([self.encoder_h_context, x], dim = -1)
        output, hc = self.rnn(x, h)
        output = torch.concat([self.encoder_h_context, output], dim = -1)
        output = self.f(output)
        return output, hc

encoder = Encoder(embedding_tensor).to(device)
decoder = Decoder(embedding_tensor).to(device)

loss_function = nn.CrossEntropyLoss()
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr = 0.02)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr = 0.02)
epoch = 100
prev_acc = 0
acc_cnt = 0

for e in range(epoch) :
    encoder.train()
    decoder.train()
    loss_sum = 0
    for x, t in train_dataloader :
        encoder_output, encoder_h = encoder(x)
        y, _ , _ = decoder(encoder_output, encoder_h, t)

        loss = loss_function(y.reshape(-1, y.shape[-1]), t.reshape(-1))
        loss_sum += loss.item()

        decoder_optimizer.zero_grad()
        encoder_optimizer.zero_grad()
        loss.backward()
        decoder_optimizer.step()
        encoder_optimizer.step()
    loss_sum /= len(train_dataloader)
    # print(f"epoch {e+1} | loss {loss_sum}")

    encoder.eval()
    decoder.eval()
    correct = 0
    total = 0
    for x, t in test_dataloader :
        encoder_output, encoder_h = encoder(x)
        y, _ , _ = decoder(encoder_output, encoder_h, None)
        correct += (y.argmax(dim = -1) == t).sum().item()
        total += len(x) * 4
    acc = correct / total

    if acc <= prev_acc :
        acc_cnt += 1
    else :
        acc_cnt = 0
        prev_acc = acc
    print(f"epoch {e+1} | loss {loss_sum} | acc {acc} | cnt {acc_cnt}")
    if acc_cnt >= 5 :
        print("train stopped")
        break

epoch 1 | loss 1.5157342875003814 | acc 0.4827 | cnt 0
epoch 2 | loss 1.0858748070895672 | acc 0.55415 | cnt 0
epoch 3 | loss 0.9774710930883884 | acc 0.582175 | cnt 0
epoch 4 | loss 0.9210067439079285 | acc 0.5964 | cnt 0
epoch 5 | loss 0.8739124171435833 | acc 0.603425 | cnt 0
epoch 6 | loss 0.8412023101747036 | acc 0.6202 | cnt 0
epoch 7 | loss 0.7882262888550758 | acc 0.637075 | cnt 0
epoch 8 | loss 0.6777412127703428 | acc 0.6938 | cnt 0
epoch 9 | loss 0.575065116584301 | acc 0.736525 | cnt 0
epoch 10 | loss 0.49395442083477975 | acc 0.77275 | cnt 0
epoch 11 | loss 0.4365507024526596 | acc 0.793 | cnt 0
epoch 12 | loss 0.3973407522588968 | acc 0.804675 | cnt 0
epoch 13 | loss 0.3673285325616598 | acc 0.83605 | cnt 0
epoch 14 | loss 0.3366508091241121 | acc 0.83435 | cnt 1
epoch 15 | loss 0.3213544716313481 | acc 0.8364 | cnt 0
epoch 16 | loss 0.3085519750416279 | acc 0.8492 | cnt 0
epoch 17 | loss 0.28736987464129926 | acc 0.843075 | cnt 1
epoch 18 | loss 0.27397551234811546 | acc

In [11]:
encoder = encoder.to("cpu")
decoder = decoder.to("cpu")

torch.save(encoder, "num_encoder.pt")
torch.save(decoder, "num_decoder.pt")